In [ ]:
import re
import os


In [ ]:
with open('./疫情通报十则.md','r') as f:
    tongbao=f.read()

tongbao

# 爬取关键信息：
- 日期
- 新增确诊病xx例
- 当日新增境外输入病xx例

累计统计：
- 累计出院病例xx例
- 累计死亡病例xx例
- 累计报告确诊病例xx例
- 累计追踪到密切接触者xx人
- 当日新增治愈出院病例xx例
- 现有疑似病例xx例
- 累计追踪到密切接触者xxx人
- 尚在医学观察的密切接触者xx人

步骤：
- 找到字符串位置
- 提取字符串结构(*highlight:注意中英文的标点符号！*)
- 修改为正则表达式
- 使用合适的函数和加flag
- 分组

In [ ]:

# 观察字符串，找到位置

# 新增确诊病例313例。其中境外输入病例64例

# 现有确诊病例4456例（其中重症病例16例），累计治愈出院病例245486例，累计死亡病例5226例，累计报告确诊病例255168例，无现有疑似病例。累计追踪到密切接触者6606769人，尚在医学观察的密切接触者395576人。

# 现有确诊病例2997例（其中重症病例41例），累计治愈出院病例242070例，累计死亡病例5226例，累计报告确诊病例250293例，无现有疑似病例。累计追踪到密切接触者5993958人，尚在医学观察的密切接触者155222人。

re.findall(pattern='【截至(\d+月\d+日)24时.*?新增确诊病例(\d+)例。其中境外输入病例(\d+)例',string=tongbao,flags=re.S)
re.findall(pattern='现有确诊病例(\d+)例（其中重症病例(\d+)例），累计治愈出院病例(\d+)例，累计死亡病例(\d+)例，累计报告确诊病例(\d+)例，现有疑似病例(\d+)例。累计追踪到密切接触者(\d+)人，尚在医学观察的密切接触者(\d+)人。',string=tongbao,flags=re.S)

# what happened?


 `[\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b] `该表达式可以识别出： 。 ； ， ： “ ”（ ） 、 ？ 《 》 这些标点符号。

`\w`匹配的仅仅是中文，数字，字母

匹配中文字符的正则表达式： `[\u4e00-\u9fa5]`

匹配双字节字符(包括汉字在内)：`[^\x00-\xff]`

- “现有疑似病例”可能有，可能没有。

In [ ]:

# 把英文标点符号全都转为中文括号
tongbao=tongbao.replace('(','（')
tongbao=tongbao.replace(')','）')

re.findall(pattern='现有确诊病例(\d+)例（其中重症病例(\d+)例），累计治愈出院病例(\d+)例，累计死亡病例(\d+)例，累计报告确诊病例(\d+)例，(无)?现有疑似病例(\d+例)?。累计追踪到密切接触者(\d+)人，尚在医学观察的密切接触者(\d+)人。',string=tongbao,flags=re.S)

# 用【组】将数据提取出来


# GB格式引文目录解析

引文目录，结构清晰的数据。

 ## 这次来使用`pd.Series.str.extract()`完成提取

In [ ]:
import pandas as pd


data=pd.read_table('./GB格式引文目录.md',sep='\n',header=None)
data

提取数据：
- 作者
- 文章名
- 文献类型
- 文章来源
- 发表年限

In [ ]:
data[0][0]

## Highligts:
- 对转义字符的使用
- 正则语法：
  - `{m}`
  - `.`
  - `.*?`不贪心匹配
  - `.*`贪心匹配
  - unicode字符匹配

In [ ]:
exper=re.compile(pattern='([,\s\u4e00-\u9fa5]+)\.([^\.]*?)\[(.*?)\]\.([,\s\u4e00-\u9fa5\(\)]+),.?(\d{4})')
exper.findall(data[0][2])

In [ ]:
data[0].str.extract(exper)

## 为数据分组命名

In [ ]:
exper=re.compile(pattern='(?P<作者>[,\s\u4e00-\u9fa5]+)\.(?P<标题>[^\.]*?)\[(?P<文献类型>.*?)\]\.(?P<文献来源>[,\s\u4e00-\u9fa5\(\)]+), (?P<年份>\d{4})')
data[0].str.extract(exper)